In [1]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, BertModel, BertForSequenceClassification, BertTokenizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Concatenate, GlobalAveragePooling1D
import torch
from torchsummary import summary
import tensorflow as tf
import datasets
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight
from time import time
import gc

In [2]:
factuals = ["Who actually gives a stuff about what the Qur'an says?",
           "All the immature homophobic remarks makes you look uneducated",
           "Are you completely devoid of empathy and imagination?",
           "Christians are morons. Some people will believe (in) anything."]

In [3]:
cf_more_aggr = ["Who the hell cares about that garbage Qur'an anyway?",
                  "All the ignorant homophobic drivel just proves you're a complete moron.",
                  "Do you have a brain at all? Your lack of empathy and imagination is beyond pathetic.",
                  "All Christians are brainless idiots. People like you will swallow any load of crap without questioning it."]

In [4]:
cf_less_aggr = ["It's valid to explore different perspectives on the Qur'an. Let's foster open dialogue and respect diverse opinions.",
                  "Encouraging respectful discussions helps foster a more inclusive environment. Let's focus on understanding and learning from diverse perspectives.",
                  "Considering different perspectives allows for a richer understanding. Let's engage in empathetic conversations that promote mutual understanding and growth.",
                  "Let's remember that people hold diverse beliefs. It's essential to engage in respectful conversations rather than dismissing others based on faith."]

In [5]:
cf_pos = ["Many people appreciate the valuable insights shared in the Qur'an. It's truly a source of wisdom for those seeking knowledge and understanding.",
          "Your open-mindedness and inclusive attitude reflect a highly educated and mature perspective. It's wonderful to see acceptance and respect for diversity.",
          "Your capacity for empathy and imagination is truly remarkable, fostering a positive and understanding environment.",
          "People of all faiths possess unique wisdom and beliefs. The diversity of perspectives enriches our understanding and fosters a world of acceptance."]

In [6]:
cf_neg = ["Who actually cares about what the Qur'an says? It's not like wisdom and guidance matter in a world oblivious to profound insights.",
          "All the immature homophobic remarks make you look exceptionally ignorant. It's astonishing how some choose prejudice over enlightenment.",
          "Are you fully saturated with apathy and banality? It's astounding how some lack the depth to comprehend empathy or creativity.",
          "Christians are geniuses. Some people will doubt anything. Embracing ignorance must feel like a blissful choice for those seeking intellectual oblivion."]

In [7]:
counterfactuals = cf_more_aggr + cf_less_aggr + cf_pos + cf_neg

# Causal modules

In [8]:
#roBERTa models
sent_MOD = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
aggr_MOD = f"cardiffnlp/twitter-roberta-base-offensive"

#Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(sent_MOD)
sent_class_mod = TFAutoModelForSequenceClassification.from_pretrained(sent_MOD)
aggr_class_mod = TFAutoModelForSequenceClassification.from_pretrained(aggr_MOD)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [9]:
def Compute_Caus_Outs(tokenizer,sent_mod,aggr_mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out_1 = []
  batch_out_2 = []
  for batch in batches:
      encoded_inputs = tokenizer(batch,padding=True,return_tensors='tf')
      # Compute the outputs of the modules as numpy arrays.
      emb_1 = sent_mod(encoded_inputs)[0]
      scores_1 = []
      for i in range(emb_1.shape[0]):
            scores_1.append(softmax(emb_1[i].numpy()))
      scores_1 = np.array(scores_1)
      batch_out_1.append(scores_1)

      emb_2 = aggr_mod(encoded_inputs)[0]
      scores_2 = []
      for i in range(emb_2.shape[0]):
            scores_2.append(softmax(emb_2[i].numpy()))
      scores_2 = np.array(scores_2)
      batch_out_2.append(scores_2)
    
      encoded_inputs = None
      emb_1 = None
      emb_2 = None
      gc.collect() 
    
  outs_sent = np.concatenate(batch_out_1,axis=0)
  outs_aggr = np.concatenate(batch_out_2,axis=0)  

  return outs_sent,outs_aggr

## Counterfactual evaluation

In [10]:
def print_causal_result(text,sent,aggr):
    print("Text:")
    print(text)
    print()
    print("Sentiment")
    print("Negative:",sent[0])
    print("Neutral:",sent[1])
    print("Positive:",sent[2])
    print()
    print("Aggression")
    print("Not offensive:",aggr[0])
    print("Offensive:",aggr[1])
    
def print_causal_results(texts,sents,aggrs):
    for i in range(len(texts)):
        print_causal_result(texts[i],sents[i],aggrs[i])
        print()
        print()

### Causal factors: Sentiment and aggression

#### Factuals

In [11]:
batch_size = 32

class_fact_sent,class_fact_aggr = Compute_Caus_Outs(tokenizer,sent_class_mod,aggr_class_mod,factuals,batch_size)
print_causal_results(factuals,class_fact_sent,class_fact_aggr)

Text:
Who actually gives a stuff about what the Qur'an says?

Sentiment
Negative: 0.33747652
Neutral: 0.30590466
Positive: 0.35661882

Aggression
Not offensive: 0.55399185
Offensive: 0.44600818


Text:
All the immature homophobic remarks makes you look uneducated

Sentiment
Negative: 0.3257707
Neutral: 0.3080832
Positive: 0.36614612

Aggression
Not offensive: 0.14936976
Offensive: 0.8506303


Text:
Are you completely devoid of empathy and imagination?

Sentiment
Negative: 0.31708872
Neutral: 0.3110881
Positive: 0.37182313

Aggression
Not offensive: 0.6127301
Offensive: 0.38726997


Text:
Christians are morons. Some people will believe (in) anything.

Sentiment
Negative: 0.3379351
Neutral: 0.31368178
Positive: 0.34838316

Aggression
Not offensive: 0.1583086
Offensive: 0.84169143




#### Counterfactuals

In [12]:
class_cfact_sent,class_cfact_aggr = Compute_Caus_Outs(tokenizer,sent_class_mod,aggr_class_mod,counterfactuals,batch_size)
print_causal_results(counterfactuals,class_cfact_sent,class_cfact_aggr)

Text:
Who the hell cares about that garbage Qur'an anyway?

Sentiment
Negative: 0.31999543
Neutral: 0.32466948
Positive: 0.35533506

Aggression
Not offensive: 0.14140826
Offensive: 0.85859174


Text:
All the ignorant homophobic drivel just proves you're a complete moron.

Sentiment
Negative: 0.33129492
Neutral: 0.30829602
Positive: 0.36040902

Aggression
Not offensive: 0.07762642
Offensive: 0.9223736


Text:
Do you have a brain at all? Your lack of empathy and imagination is beyond pathetic.

Sentiment
Negative: 0.31620136
Neutral: 0.3145087
Positive: 0.36928993

Aggression
Not offensive: 0.210876
Offensive: 0.78912395


Text:
All Christians are brainless idiots. People like you will swallow any load of crap without questioning it.

Sentiment
Negative: 0.32900533
Neutral: 0.31449714
Positive: 0.35649747

Aggression
Not offensive: 0.07743224
Offensive: 0.9225678


Text:
It's valid to explore different perspectives on the Qur'an. Let's foster open dialogue and respect diverse opinions.



# Hate models

## Load data labels

In [8]:
#FRENK dataset
ds = datasets.load_dataset("classla/FRENK-hate-en","binary")
label_train = np.array(ds["train"]["label"])

Found cached dataset frenk-hate-en (C:/Users/Pedro/.cache/huggingface/datasets/classla___frenk-hate-en/binary/0.0.0/81373cea138e2fc4156010c45a19880095b891a74c39768895384d352bfe1ca0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# def get_hate_labels(labels):
#     hate_labels = np.zeros(len(labels))
#     for i in range(len(labels)):
#         if labels[i].any():
#             hate_labels[i] = 1
#     return hate_labels

In [10]:
# #Gab dataset
# df_train = pd.read_csv('data/ghc_train.tsv',sep='\t')
# labels = df_train[["hd","cv","vo"]].to_numpy()
# ghc_label_train = get_hate_labels(labels)
# ghc_label_train = ghc_label_train[:8404]

In [11]:
# #Twi-Red-You dataset
# df_train = pd.read_csv('data/try_train_8404.tsv',sep='\t')
# try_label_train = df_train['hate'].to_numpy()

In [12]:
# label_train = np.concatenate([frenk_label_train, ghc_label_train, try_label_train])

In [13]:
c_weights = compute_class_weight('balanced', classes=np.unique(label_train), y=label_train)
c_weights = {0:c_weights[0], 1:c_weights[1]}

## Load data embeddings

In [14]:
sent_train = np.load('outs/frenk/sent_train_outs.npy')
aggr_train = np.load('outs/frenk/aggr_train_outs.npy')
gen_train = np.load('outs/frenk/gen_train_outs.npy')

In [15]:
# ghc_sent_train = np.load('outs/ghc/sent_train_outs.npy')[:8404]
# ghc_aggr_train = np.load('outs/ghc/aggr_train_outs.npy')[:8404]
# ghc_gen_train = np.load('outs/ghc/gen_train_outs.npy')[:8404]

In [16]:
# try_sent_train = np.load('outs/try/sent_train_outs.npy')
# try_aggr_train = np.load('outs/try/aggr_train_outs.npy')
# try_gen_train = np.load('outs/try/gen_train_outs.npy')

In [17]:
# sent_train = np.concatenate([frenk_sent_train, ghc_sent_train, try_sent_train],axis=0)
# aggr_train = np.concatenate([frenk_aggr_train, ghc_aggr_train, try_aggr_train],axis=0)
# gen_train = np.concatenate([frenk_gen_train, ghc_gen_train, try_gen_train],axis=0)

In [18]:
conc_train = np.concatenate([sent_train,aggr_train,gen_train],axis=1)

## Peace modules

In [19]:
#roBERTa models
sent_MOD = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
aggr_MOD = f"cardiffnlp/twitter-roberta-base-offensive"
gen_MOD = f"roberta-base"

#Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(sent_MOD)
sent_mod = TFAutoModel.from_pretrained(sent_MOD)
aggr_mod = TFAutoModel.from_pretrained(aggr_MOD)
gen_mod = TFAutoModel.from_pretrained(gen_MOD)

batch_size = 32

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel

In [20]:
def Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out_1 = []
  batch_out_2 = []
  batch_out_3 = []
  for batch in batches:
      encoded_inputs = tokenizer(batch,padding=True,return_tensors='tf')
      # Compute the outputs of the modules as numpy arrays.
      emb_1 = sent_mod(encoded_inputs)[0]
      pool_1 = GlobalAveragePooling1D()(emb_1).numpy()
      batch_out_1.append(pool_1)

      emb_2 = aggr_mod(encoded_inputs)[0]
      pool_2 = GlobalAveragePooling1D()(emb_2).numpy()
      batch_out_2.append(pool_2)

      emb_3 = gen_mod(encoded_inputs)[0]
      pool_3 = GlobalAveragePooling1D()(emb_3).numpy()
      batch_out_3.append(pool_3)
    
      encoded_inputs = None
      emb_1 = None
      emb_2 = None
      emb_3 = None
      gc.collect() 
    
  outs_1 = np.concatenate(batch_out_1,axis=0)
  outs_2 = np.concatenate(batch_out_2,axis=0)
  outs_3 = np.concatenate(batch_out_3,axis=0)  

  return outs_1,outs_2,outs_3

## Counterfactual evaluation

In [21]:
def print_hate_result(text,hate):
    print("Text:")
    print(text)
    print("Hate prob.:",hate[0])
    
def print_hate_results(texts,hates):
    for i in range(len(texts)):
        print_hate_result(texts[i],hates[i])
        print()
        print()

In [22]:
n_avg = 10

### PEACE model

In [23]:
def evaluate_texts_PEACE(texts,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    out_sent,out_aggr,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size)
    conc = np.concatenate([out_sent,out_aggr,out_gen],axis=1)
    probs = model.predict(conc)
    return probs

In [24]:
f_hate = []
cf_hate = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_train, label_train, class_weight=c_weights, epochs=5, batch_size=32)
    f_hate.append(evaluate_texts_PEACE(factuals,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size))
    cf_hate.append(evaluate_texts_PEACE(counterfactuals,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size))
f_hate = np.mean(f_hate,axis=0)
cf_hate = np.mean(cf_hate,axis=0)

Epoch 1/5
263/263 [==============================] - 2s 4ms/step - loss: 0.5385 - accuracy: 0.7228
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4956 - accuracy: 0.7430
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4760 - accuracy: 0.7499
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4633 - accuracy: 0.7599
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4521 - accuracy: 0.7664
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5387 - accuracy: 0.7157
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4971 - accuracy: 0.7374
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4774 - accuracy: 0.7508
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4591 - accuracy: 0.7624
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4469 - accuracy: 0.7694
Epoch 1/5


In [25]:
print(f_hate)

[[0.44294176]
 [0.91992617]
 [0.59891474]
 [0.959857  ]]


#### Factuals

In [26]:
print_hate_results(factuals,f_hate)

Text:
Who actually gives a stuff about what the Qur'an says?
Hate prob.: 0.44294176


Text:
All the immature homophobic remarks makes you look uneducated
Hate prob.: 0.91992617


Text:
Are you completely devoid of empathy and imagination?
Hate prob.: 0.59891474


Text:
Christians are morons. Some people will believe (in) anything.
Hate prob.: 0.959857




#### Counterfactuals

In [27]:
print_hate_results(counterfactuals,cf_hate)

Text:
Who the hell cares about that garbage Qur'an anyway?
Hate prob.: 0.82672805


Text:
All the ignorant homophobic drivel just proves you're a complete moron.
Hate prob.: 0.978858


Text:
Do you have a brain at all? Your lack of empathy and imagination is beyond pathetic.
Hate prob.: 0.9820507


Text:
All Christians are brainless idiots. People like you will swallow any load of crap without questioning it.
Hate prob.: 0.98218334


Text:
It's valid to explore different perspectives on the Qur'an. Let's foster open dialogue and respect diverse opinions.
Hate prob.: 0.012865147


Text:
Encouraging respectful discussions helps foster a more inclusive environment. Let's focus on understanding and learning from diverse perspectives.
Hate prob.: 0.015143883


Text:
Considering different perspectives allows for a richer understanding. Let's engage in empathetic conversations that promote mutual understanding and growth.
Hate prob.: 0.015358342


Text:
Let's remember that people hold diverse

### General model

In [28]:
def evaluate_texts_gen(texts,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    _,_,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size)
    probs = model.predict(out_gen)
    return probs

In [29]:
f_hate = []
cf_hate = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(gen_train, label_train, class_weight=c_weights, epochs=5, batch_size=32)
    f_hate.append(evaluate_texts_gen(factuals,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size))
    cf_hate.append(evaluate_texts_gen(counterfactuals,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size))
f_hate = np.mean(f_hate,axis=0)
cf_hate = np.mean(cf_hate,axis=0)

Epoch 1/5
263/263 [==============================] - 2s 4ms/step - loss: 0.5918 - accuracy: 0.6627
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.5283 - accuracy: 0.7217
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5093 - accuracy: 0.7335
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4908 - accuracy: 0.7438
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4730 - accuracy: 0.7538
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5917 - accuracy: 0.6687
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5346 - accuracy: 0.7170
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5109 - accuracy: 0.7339
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4966 - accuracy: 0.7402
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4751 - accuracy: 0.7521
Epoch 1/5


#### Factuals

In [30]:
print_hate_results(factuals,f_hate)

Text:
Who actually gives a stuff about what the Qur'an says?
Hate prob.: 0.16510433


Text:
All the immature homophobic remarks makes you look uneducated
Hate prob.: 0.9250064


Text:
Are you completely devoid of empathy and imagination?
Hate prob.: 0.57922256


Text:
Christians are morons. Some people will believe (in) anything.
Hate prob.: 0.6008378




#### Counterfactuals

In [31]:
print_hate_results(counterfactuals,cf_hate)

Text:
Who the hell cares about that garbage Qur'an anyway?
Hate prob.: 0.50020856


Text:
All the ignorant homophobic drivel just proves you're a complete moron.
Hate prob.: 0.959298


Text:
Do you have a brain at all? Your lack of empathy and imagination is beyond pathetic.
Hate prob.: 0.88274926


Text:
All Christians are brainless idiots. People like you will swallow any load of crap without questioning it.
Hate prob.: 0.8992152


Text:
It's valid to explore different perspectives on the Qur'an. Let's foster open dialogue and respect diverse opinions.
Hate prob.: 0.044760793


Text:
Encouraging respectful discussions helps foster a more inclusive environment. Let's focus on understanding and learning from diverse perspectives.
Hate prob.: 0.048087917


Text:
Considering different perspectives allows for a richer understanding. Let's engage in empathetic conversations that promote mutual understanding and growth.
Hate prob.: 0.03621191


Text:
Let's remember that people hold diverse 

### HateBERT

In [32]:
train_emb = np.load('outs/frenk/HateBERT_train_emb.npy')
#ghc_train_emb = np.load('outs/ghc/HateBERT_train_emb.npy')[:8404]
#try_train_emb = np.load('outs/try/HateBERT_train_emb.npy')
#train_emb = np.concatenate([frenk_train_emb,ghc_train_emb,try_train_emb],axis=0)

In [33]:
model_dir = "HateBERT_hateval"
HateBERT = BertModel.from_pretrained(
    model_dir # Use pre-trained model from its directory, change this to use a pre-trained model from bert
)
HateBERT.eval()
HateBERT_tok = BertTokenizer.from_pretrained(model_dir)

Some weights of the model checkpoint at HateBERT_hateval were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
def Comp_Embed_HateBERT(tokenizer,mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out = []

  with torch.no_grad():
      for batch in batches:
          encoded_inputs = tokenizer(batch,padding=True,return_tensors='pt')
          # Compute the output of the module as a numpy array.

          emb = mod(**encoded_inputs)[0]
          tf_emb = tf.convert_to_tensor(emb.numpy())  
          pool = GlobalAveragePooling1D()(tf_emb).numpy()
          batch_out.append(pool)

          encoded_inputs = None
          tf_emb = None
          gc.collect() 
    
  outs = np.concatenate(batch_out,axis=0)
  return outs

In [35]:
def evaluate_texts_HateBERT(texts,model,tokenizer,HateBERT,batch_size):
    out = Comp_Embed_HateBERT(tokenizer,HateBERT,texts,batch_size)
    probs = model.predict(out)
    return probs

In [36]:
f_hate = []
cf_hate = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(train_emb, label_train, class_weight=c_weights, epochs=5, batch_size=32)
    f_hate.append(evaluate_texts_HateBERT(factuals,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size))
    cf_hate.append(evaluate_texts_HateBERT(counterfactuals,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size))
f_hate = np.mean(f_hate,axis=0)
cf_hate = np.mean(cf_hate,axis=0)

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5520 - accuracy: 0.7012
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4920 - accuracy: 0.7477
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4482 - accuracy: 0.7752
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4017 - accuracy: 0.8069
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3484 - accuracy: 0.8381
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.5542 - accuracy: 0.7055
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4965 - accuracy: 0.7457
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4534 - accuracy: 0.7771
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4083 - accuracy: 0.8034
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3592 - accuracy: 0.8344
Epoch 1/5


#### Factuals

In [37]:
print_hate_results(factuals,f_hate)

Text:
Who actually gives a stuff about what the Qur'an says?
Hate prob.: 0.5194335


Text:
All the immature homophobic remarks makes you look uneducated
Hate prob.: 0.9707793


Text:
Are you completely devoid of empathy and imagination?
Hate prob.: 0.7609297


Text:
Christians are morons. Some people will believe (in) anything.
Hate prob.: 0.9770282




#### Counterfactuals

In [38]:
print_hate_results(counterfactuals,cf_hate)

Text:
Who the hell cares about that garbage Qur'an anyway?
Hate prob.: 0.9036473


Text:
All the ignorant homophobic drivel just proves you're a complete moron.
Hate prob.: 0.99675894


Text:
Do you have a brain at all? Your lack of empathy and imagination is beyond pathetic.
Hate prob.: 0.9085639


Text:
All Christians are brainless idiots. People like you will swallow any load of crap without questioning it.
Hate prob.: 0.9839948


Text:
It's valid to explore different perspectives on the Qur'an. Let's foster open dialogue and respect diverse opinions.
Hate prob.: 0.14451924


Text:
Encouraging respectful discussions helps foster a more inclusive environment. Let's focus on understanding and learning from diverse perspectives.
Hate prob.: 0.020944662


Text:
Considering different perspectives allows for a richer understanding. Let's engage in empathetic conversations that promote mutual understanding and growth.
Hate prob.: 0.13122372


Text:
Let's remember that people hold diverse b